# Integer numbers
$x\in\mathbb{Z}$
## Python disclaimer
Originally computers we designed to solve computational tasks (over numbers). CPU Architectures (e.g. x64) encount the fact that operations over $\mathbb{Z}$ and $\mathbb{R}$ should be predictibly fast: they [fix number representation length](https://en.wikipedia.org/wiki/Processor_register#Size), sacrifising precision or edge cases.

Most programming languages reuse CPU number representation, defined in [standards](https://en.wikipedia.org/wiki/IEEE_754-2008_revision): `short, int, long, float/real, double` in majority of languages are binary interchangable.

Python is following this strategy for $\mathbb{R}$ (`float`) data, but disrespects for $\mathbb{Z}$ (`int`).

In [20]:
import sys
x = 10000000000000000000000000000000000000000000000000000000000000000000000000000000000
y = 20
a = 20.
b = 0.000000000000000000000000000000000000000000000000000000000000000000000000000000001
print(f'{type(x)}\t x ~ {sys.getsizeof(x)} B; y ~ {sys.getsizeof(y)} B')
print(f'{type(a)}\t a ~ {sys.getsizeof(a)} B; b ~ {sys.getsizeof(b)} B')

<class 'int'>	 x ~ 64 B; y ~ 28 B
<class 'float'>	 a ~ 24 B; b ~ 24 B


This is how this difference can be observed

In [24]:
print("x^4 =", x**4)
print("b^4 =", b**4, b**4 == 0)

x^4 = 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
b^4 = 0.0 True


To make python work as other languages we will use [native types](https://stackoverflow.com/questions/19716315/how-to-create-a-fixed-size-unsigned-integer-in-python).

In [72]:
from ctypes import c_char, c_byte, c_ushort, c_int16, c_uint32, c_int32, c_float

i = c_int32(2000000000)
f = c_float()
f.value = .4

## Signed integer numbers

Binary:

47<sub>10</sub> = 
    **1** * 2<sup>5</sup>
    + 0 * 2<sup>4</sup>
    + **1** * 2<sup>3</sup>
    + **1** * 2<sup>2</sup>
    + **1** * 2<sup>1</sup> 
    + **1** * 2<sup>0</sup> = 101111<sub>2</sub> = 00101111 (char) = 00000000 00000000 00000000 00101111 (int)
    
```
[high bit, most significant bit = msb] 00000000 00000000 00000000 0 0 1 0 1 1 1 1 [low bit, lsb]
                                      31                          7 6 5 4 3 2 1 0
```

In [48]:
i = c_int32(0b101111)
print(i.value)

47


## What about negatives?

### Sign-magnitude
Use the highes bit of type to store negative sign (1 ~ -)

What about simple arithmetics? Two zeros?

### One's complement

`-00101111` => `11010000`

But still 2 zeros. 

And what about arithmetics?

### Two's complement

... for negative integers, the absolute value of the integer is equal to "the magnitude of the complement of the (n-1)-bit binary pattern plus one" (hence called 2's complement).

`1B -13: ~(0 000 1101 - 1) -> ~(0 000 1100) -> (1 111 0011)`

`2B -13: ~(0 0000000 0000 1101 - 1) -> ~(0 0000000 0000 1100) -> (1 1111111 1111 0011)`

Sum:
```
    1111 0011
  + 0000 1101
    ---------
   10000 0000
```

In [61]:
b = c_byte(0b11110011)
s = c_int16(0b1111111111110011)
print(b.value, s.value)

-13 -13


#### Problems

1. What will be `-128` in 1-byte representation?
2. What is decimal for `01000001`?
3. What is decimal for `10000001`?

## Real numbers

In [65]:
f = -1.05e+2
print(f)

-105.0


We will speak about 1-precision numbers (`float`). Doubles are described by the same standard and differ only in positions.

For floating point value 4 bytes are used as follows:
1. The most significant bit is the **sign bit** (S), with 0 for positive numbers and 1 for negative numbers.
2. The following 8 bits represent **exponent** (E).
3. The remaining 23 bits represents **fraction** (F).

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Float_example.svg/885px-Float_example.svg.png)

### Example
`1  1000 0001  011 0000 0000 0000 0000 0000`, with:
- S = 1
- E = 1000 0001 = 129<sub>10</sub>
- F = 011 0000 0000 0000 0000 0000

1. In the normalized form, the actual fraction F is normalized with an implicit leading 1 in the form of `1.F`.
2. In normalized form, the actual exponent is `E-127` (-1024 for double)

Thus:

(-1) * (1 + 1/4 + 1/8) * 2<sup>(129 - 127)</sup> = -1.375 * 4 = -5.5
    

In [78]:
import ctypes
i = c_uint32(0b1_1000_0001_011_0000_0000_0000_0000_0000)
fp = ctypes.cast(ctypes.addressof(i), ctypes.POINTER(c_float))
f = fp[0]
print(f)

-5.5


Normalized form has a serious problem, with an implicit leading `1.` for the fraction, it cannot represent the number `0`.

### Special values and subnormal (denormalized) numbers

There are 2 special cases of exponent.

`E = 0000 0000`

1. If `F = 0`, then the number is considered ±0.
2. If `F > 0`, then this is **subnormal** number. Subnormal numbers processed differently: `value = S * 0.F * 2^(-126)`. There are also flush-to-zero (FTZ) and denormals-are-zero (DAZ) CPU flags.

E.g.

`1  0000 0000  011 0000 0000 0000 0000 0000`, with:
- S = 1
- E = 0000 0000
- F = 011 0000 0000 0000 0000 0000

`-0.375×2^(-126) ~ -4.4×10^(-39)`

In [83]:
i = c_uint32(0b1_0000_0000_011_0000_0000_0000_0000_0000)
fp = ctypes.cast(ctypes.addressof(i), ctypes.POINTER(c_float))
f = fp[0]
print(f)

-4.408103815583578e-39


`E = 1111 1111`

1. 

https://docs.google.com/document/d/1bDwyXvjsseviSKq8sJo5ObEen02XuX4WDnOoGX3W18w/edit#

Представление числа. Целые, длинные, с плавающей запятой. Особенности использования в приложении к точности и скорости вычислений.
Варианты представления чисел.
Арифметическое переполнение.
Неравномерность действительных чисел.
Особенности сложения.

Числа с плавающей запятой (точкой)
The most significant bit is the sign bit (S), with 0 for positive numbers and 1 for negative numbers.
The following 8 bits represent exponent (E).
The remaining 23 bits represents fraction (F).
1 1000 0001 011 0000 0000 0000 0000 0000, with:
S = 1
E = 1000 0001
F = 011 0000 0000 0000 0000 0000
In the normalized form, the actual fraction F is normalized with an implicit leading 1 in the form of 1.F
1.011 0000 0000 0000 0000 0000 = 1 + 1×2^-2 + 1×2^-3 = 1.375.
S=1, this is a negative number, i.e., -1.375
In normalized form, the actual exponent is E-127 (-1024 for double). Actual exponent of -127 to 128
E-127 = 1000 00012 - 12710 = 129-127=2.  ⇒ -1.375×2^2=-5.5
Normalized form has a serious problem, with an implicit leading 1 for the fraction, it cannot represent the number zero.
Согласно стандарту, если порядок равен своему минимальному значению (все его биты — нули, а истинное значение порядка равно его сдвигу) и все биты мантиссы равны нулю, то это  ±0. Если же мантисса не равна нулю, то это число с порядком, на единицу большим минимального (все биты порядка, кроме младшего — нули) и данной мантиссой, целая часть которой считается равной нулю, а не единице. Ввиду сложности денормализованные числа крайне редко реализуют на аппаратном уровне - вместо этого используются программные реализации, работающие значительно медленнее.
Иногда они просто игнорируются. При этом используются два простых механизма, получивших название Flush-to-zero (FTZ) и Denormals-are-zero (DAZ).
В версии стандарта IEEE 754-2008 денормализованные числа (denormal или denormalized numbers) были переименованы в subnormal numbers, то есть в числа, меньшие "нормальных"
For E=0, the numbers are in the de-normalized form. An implicit leading 0 (instead of 1) is used for the fraction; and the actual exponent is always -126. Hence, the number zero can be represented with E=0 and F=0 (because 0.0×2^-126=0).
Положительный нуль +0 и отрицательный нуль −0. число считается нулём, если все его биты, кроме знакового, равны нулю. При этом в зависимости от значения бита знака ноль может быть как положительным, так и отрицательным.
We can also represent very small positive and negative numbers in de-normalized form with E=0. For example, if S=1, E=0, and F=011 0000 0000 0000 0000 0000. The actual fraction is 
0.011...2 = 1×2^(-2)+1×2^(-3)=0.375
Since S=1, it is a negative number. With E=0, the actual exponent is -126. Hence the number is 
-0.375×2^(-126) = -4.4×10^(-39)
which is an extremely small negative number (close to zero).
